# ノートブックの概要
EDA及び、前処理がちゃんと動きそうかの確認のためのノートブック。以下のことを行っている
- データ読み込んでみる
- 時系列データを可視化
- 時間軸をどこで区切るべきか（年？年度？）
- 緩やかな変動を捉える（prophet）
  - トレンドとの残差を確認
- 引っ越し件数が0の部分（＝休業日と判明）
- clientフラグについて
- 料金区分（午前、午後）
  - 料金区分（price_am, price_pm）が3以上のものを詳しく見てみる
- 数値変換を試す

# ライブラリのインポート

In [ ]:
import polars as pl
import polars.selectors as cs
pl.Config.set_tbl_cols(100)
pl.Config.set_tbl_width_chars(200)

import numpy as np
from scipy import stats

import matplotlib.pyplot as plt
%matplotlib inline
import japanize_matplotlib

import seaborn as sns

import plotly
plotly.offline.init_notebook_mode()  # github pages 対応
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from datetime import date
import matplotlib.dates as mdates

import jpholiday

from prophet import Prophet

# データ読み込んでみる

In [ ]:
train = pl.read_csv("../data/input/train.csv")
test = pl.read_csv("../data/input/test.csv")
sample_submit = pl.read_csv("../data/input/sample_submit.csv", has_header=False)

print("train\n", train)
print("test\n", test)
print("sample_submit\n", sample_submit)

データの読み方について  

例えば  
│datetime   | y   | client | close | price_am | price_pm |
| ---- | ---- | ---- | ---- | ---- | ---- |
│2016-03-28 | 86  | 1      | 0     | 4        | 4        │  

といったとき。  
これは、  
日時「2016-03-28」について、この日は  
合計引っ越し数は「86件」  
法人が絡む特殊な引越し日フラグは「あり」  
この日の午前料金区分は「4」  
この日の午後料金区分は「4」  
といった意味っぽい  

In [ ]:
# IDとして使用されているdatetimeカラムを文字列型と日付型に分離
# カラムの役割を分散することでコードエラーを発生しにくくする
train = train.insert_column(0, train["datetime"].alias("id")).with_columns(pl.col("datetime").str.strptime(dtype=pl.Date))
test = test.insert_column(0, test["datetime"].alias("id")).with_columns(pl.col("datetime").str.strptime(dtype=pl.Date))

print(train.head())

In [ ]:
# 訓練データの散布図行列
g = sns.pairplot(
    train.select(cs.exclude("id")).to_pandas(), 
    plot_kws={"alpha": 0.5, "s": 10}, 
    diag_kws={"alpha": 0.7, "bins": 10},
)
g.figure.set_size_inches(8, 8)
g.figure.suptitle("特徴量間のペアプロット", y=1.02, fontsize=14)
plt.show()

# 時系列データを可視化

In [ ]:
# 歪度と尖度を計算
skew = stats.skew(train["y"])
kurt = stats.kurtosis(train["y"])

# 可視化
fig, axes = plt.subplots(
    nrows=1, ncols=2,
    height_ratios=[1], width_ratios=[2, 1],
    figsize=(12, 4),
    constrained_layout=True,
)
fig.suptitle("引っ越し数yの推移と分布")

axes[0].set_title("推移")
sns.lineplot(data = train, x="datetime", y="y", ax=axes[0])

axes[1].set_title("分布")
sns.histplot(data = train, x="y", ax=axes[1])
axes[1].text(0.6, 0.8,
                f"skewness={skew:.2f}\nkurtosis={kurt:.2f}",
                fontsize=12,
                transform=axes[1].transAxes)

plt.show()
del skew, kurt

In [ ]:
# 繁忙期を抜き出して見てみる
train = train.with_columns(
    (
        ((pl.col("datetime").dt.month() == 3) | (pl.col("datetime").dt.month() == 4))
    ).alias("is_busy")
)

fig, axes = plt.subplots(
    nrows=1, ncols=2,
    height_ratios=[1], width_ratios=[2, 1],
    figsize=(12, 4),
    constrained_layout=True,
)
fig.suptitle("引っ越し数yの推移と分布（繁忙期・非繁忙期）")

axes[0].set_title("推移")
sns.lineplot(data = train, x="datetime", y="y", hue="is_busy", ax=axes[0])

axes[1].set_title("分布")
sns.histplot(data = train, x="y", hue="is_busy", ax=axes[1])

plt.show()

グラフについてのコメント  
- 引越し数は緩やかに増加傾向。ビジネス的には喜ばしいが、未来の需要予測というタスクにおいては注意が必要
  - 機械学習モデルは訓練データとテストデータの目的変数の分布が等しいことを前提としているため、これが変化すると上手く予測できない
  - 例えば、2011年データをそのまま学習させて2012年の引越し数を予測すると、予測値は実際よりも小さく見積もられてしまうものと予想される
  - この緩やかな増加傾向を別モデルで捉えてやり、その差分を機械学習モデルに学習させるのが良いだろう
- 繁忙期(3~4月)は他の時期と比べて明確に異なりそう(大きなピークとなっている)
- 引っ越し数yが0の日がある
- yの分布は非対称 → 数値変換すると良いかも？

In [ ]:
# TODO: ここらへんで一度、ydata-profillingでレポートも出力させておく？ライブラリのインストールが必要。ipywigetsも必要。
# import pandas as pd
# from ydata_profiling import ProfileReport
# data_train = pd.read_csv('titanic_train.csv')
# profile = ProfileReport(data_train, title="Profiling Report")
# profile.to_widgets()

# 時間軸をどこで区切るべきか(年？年度？)

In [ ]:
# 年ごとで分けるか？年度ごとで分けるか？
train = (
    train
    .with_columns([
        pl.col("datetime").dt.year().alias("year"),
        pl.when(pl.col("datetime").dt.month() <= 3).then(pl.col("datetime").dt.year() - 1)
        .otherwise(pl.col("datetime").dt.year())
        .alias("fy")
    ])
)
print(train.head(3))

In [ ]:
fig, axes = plt.subplots(
    nrows=2, ncols=1,
    height_ratios=[1, 1], width_ratios=[1],
    figsize=(8, 6),
    constrained_layout=True,    
)
fig.suptitle("引っ越し数yの推移　年・年度別での比較")

axes[0].set_title("年区切り")
sns.lineplot(data = train, x="datetime", y="y", hue="year", palette="deep", ax=axes[0])

axes[1].set_title("年度区切り")
sns.lineplot(data = train, x="datetime", y="y", hue="fy", palette="deep", ax=axes[1])

plt.show()

グラフへのコメント  
- 年度で分けると、繁忙期(3~4月)のピークが分割されてしまい分析が難しくなってしまう。そのため、年ごとで分けるのが適切かと

# 引っ越し件数0の部分(→休業日と判明)

In [ ]:
# y==0、close==1のデータを抽出。引っ越し件数が0件または休業日の場合
with pl.Config(tbl_rows=-1):
    print(
        "train\n", 
        train.filter(
            (pl.col("y") == 0) | (pl.col("close") == 1)
        )
    )

y=0の部分についてのコメント  
- これは…お盆のときと、年末年始(12/31~1/3)のときに全く引っ越しがないっぽいな。毎年そうみたい
- y=0の部分は機械学習に任せるのではなく、ルールベースで決めてしまったほうが良いだろう
- よく見ると、休業日(close==1)とほぼ一致している。年末年始は毎年休業日かつ引っ越し0
  - ただし、2010年、2011年のお盆は休業日じゃないけど引っ越し件数0。2012年以降はお盆も休業日になった模様？

休業日の場合についてのコメント  
- 休業日(close==1)の場合は全てy==0。これはもうルールベースで書いてしまった方が良いかな
- お盆は…どうなんだろう。今後お盆が休業日じゃなくなる日ってあるんだろうか？
  - お盆は毎年日にちが異なるので…日時でルールを決めるの難しそう
  - うーん、close==1を使ってルールを決めるのにとどめたほうがいいかな？
  - 2010年だけ例外的にお盆も営業していた？

In [ ]:
# 休業日の場合は引っ越し数0。これはルールベースで予測することとする。そのため学習からは除外

# trainからpl.col("y"==0)である部分を除外する
# 2010-08-18と2011-08-14はお盆であるものの休業日に設定されていなかったが、
# 以降の年では休業日に設定されたみたいなので学習からは取り除いておく
train = train.filter(
    (pl.col("close") != 1)
    & (pl.col("id") != "2010-08-18")
    & (pl.col("id") != "2011-08-14")
)

# test_close: testデータの中で休業日の部分(pl.col("close"==1))を抜き出し、
# その日は引っ越し数0(pl.col("y"==0))であるとしたdataframe
# 元のtestからはその部分は取り除いておく
test_close = test.filter(pl.col("close") == 1)[["id"]]
test_close = test_close.with_columns(pl.Series("y", [0.0] * len(test_close)))
test = test.filter(pl.col("close") != 1)

# ちゃんと休業日関連が消せているか確認
with pl.Config(tbl_rows=-1):
    print(
        "train\n",
        train.filter(
            (pl.col("y") == 0) | (pl.col("close") == 1)
        )
    )
    print(
        "test\n",
        test.filter(
            (pl.col("close") == 1)
        )
    )    

In [ ]:
# 分離が済んだので、close列は削除
train = train.drop("close")
test = test.drop("close")

# clientフラグについて

In [ ]:
fig, axes = plt.subplots(
    nrows=1, ncols=2,
    height_ratios=[1], width_ratios=[2, 1],
    figsize=(12, 4),
    constrained_layout=True,
)
plt.suptitle("clientフラグについて\n引越し数 y の推移と分布")

axes[0].set_title("推移")
axes[0].tick_params(axis="x", rotation=90)
sns.histplot(data=train, x="y", hue="client", ax=axes[1])

axes[1].set_title("分布")
sns.lineplot(data=train, x="datetime", y="y", hue="client", ax=axes[0])

plt.show()
print(train.filter(pl.col("client") == 1))

2014年5月からclient(法人が絡む特殊な引越し日フラグ)に関する何かが始まっているみたい  
それが何のことなのかは良くわからんけど…日にちに対してフラグが立っている  
→多分、「この日は法人からの依頼があった」ということ？yの内何件がそれだったのかはわからんけど⋯

In [ ]:
# 期間を絞り込んで表示。
fig, axes = plt.subplots(
    nrows=1, ncols=2,
    height_ratios=[1], width_ratios=[2, 1],
    figsize=(12, 4),
    constrained_layout=True,
)
plt.suptitle("clientフラグについて\n引越し数 y の推移と分布(2014-05-01以降)")

axes[0].set_title("推移")
axes[0].tick_params(axis="x", rotation=90)
sns.histplot(data=train.filter(pl.col("datetime") >= date(2014, 5, 1)), x="y", hue="client", ax=axes[1])

axes[1].set_title("分布")
sns.lineplot(data=train.filter(pl.col("datetime") >= date(2014, 5, 1)), x="datetime", y="y", hue="client", ax=axes[0])

plt.show()

print("client=1(法人あり)のときの引っ越し数yの平均",
      np.round(train.filter(pl.col("datetime") >= date(2014, 5, 1))
               .filter(pl.col("client") == 1)["y"].mean(), decimals=2)
      )
print("client=0(通常時)のときの引っ越し数yの平均",
      np.round(train.filter(pl.col("datetime") >= date(2014, 5, 1))
               .filter(pl.col("client") == 0)["y"].mean(), decimals=2)
      )

client=1のときは、そうでないときと比べて引っ越し数が通常より多めの傾向になっている  
とはいえそこまで顕著ではないようにも思うが…

# 料金区分(午前、午後)

In [ ]:
graph_color = px.colors.sequential.Magma_r
# https://oeconomicus.jp/2021/07/plotly-color-scale/

# pandas 化
df = train.to_pandas().copy()

# すべてのカテゴリ（午前/午後の和集合）で色を固定
cats_all = sorted(set(df["price_am"].unique()) | set(df["price_pm"].unique()))
colors = graph_color[:len(cats_all)]
# Plotlyのcolor_discrete_mapはキーが文字列比較になることが多いので安全のため文字列化
color_map = {str(c): colors[i] for i, c in enumerate(cats_all)}

# 描画用に文字列列を用意（凡例名/trace.nameの一致を安定させる）
df["price_am_str"] = df["price_am"].astype(str)
df["price_pm_str"] = df["price_pm"].astype(str)

# 大枠（2行×2列）
fig = make_subplots(
    rows=2, cols=2,
    row_heights=[0.5, 0.5],
    column_widths=[0.7, 0.3],
    vertical_spacing=0.15,
    horizontal_spacing=0.05,
    subplot_titles=("午前: 推移", "午前: 分布", "午後: 推移", "午後: 分布"),
)

fig.update_layout(
    width=1500,
    height=900,
    title_text="午前/午後 料金区分別<br>引越し数 y の推移と分布",
    showlegend=True,
    legend_tracegroupgap=12,
)

# 指定された行に対して、左に推移(scatter)、右に分布(histogram)を追加する関数
def add_section(colname_str: str, row: int):
    # 左：推移（scatter）
    scatter_fig = px.scatter(
        df, x="datetime", y="y", color=colname_str,
        category_orders={colname_str: [str(c) for c in cats_all]},
        color_discrete_map=color_map,
    )
    for tr in scatter_fig.data:
        # カテゴリごとに legendgroup を統一
        tr.legendgroup = tr.name                  # カテゴリ単位のグループ化
        # tr.marker.symbol = "cross"               # マーカー形状を変更 https://plotly.com/python/marker-style/
        tr.showlegend = (row == 1)               # 凡例は上段のみ表示
        fig.add_trace(tr, row=row, col=1)

    # 右：分布（histogram）
    hist_fig = px.histogram(
        df, x="y", color=colname_str,
        category_orders={colname_str: [str(c) for c in cats_all]},
        color_discrete_map=color_map,
    )
    for tr in hist_fig.data:
        tr.legendgroup = tr.name                 # 同じカテゴリ名でグルーピング
        tr.showlegend = False                    # 凡例は出さないが連動はする
        fig.add_trace(tr, row=row, col=2)

    # 軸ラベル
    fig.update_xaxes(title_text="datetime", row=row, col=1)
    fig.update_yaxes(title_text="y",        row=row, col=1)
    fig.update_xaxes(title_text="y",        row=row, col=2)
    fig.update_yaxes(title_text="count",    row=row, col=2)

# 上段: 午前
add_section("price_am_str", row=1)
# 下段: 午後
add_section("price_pm_str", row=2)

fig.show()

del df, cats_all, colors, color_map

-1は欠損。0~5で数値が高いほど料金が高い  
グラフより、yが小さいところは料金を低くしており、yが高いところは料金が高くなっていることが分かる  
説明通り、繁忙期は料金を高くして調整しようとしていることが見て取れる  
所々、料金設定ミスっているところもある？(料金設定高めなのにy低め)  
あと、2010年の料金区分のデータが欠損している？

In [ ]:
print("2010年のデータ数",
      len(train.filter(pl.col("datetime") <= date(2010, 12, 31)))
      )
print("2010年でprice_am=-1(欠損)となっているデータ数",
      len((train.filter(pl.col("datetime") <= date(2010, 12, 31))).filter(pl.col("price_am") == -1))
      )
print("2010年でprice_pm=-1(欠損)となっているデータ数",
      len((train.filter(pl.col("datetime") <= date(2010, 12, 31))).filter(pl.col("price_pm") == -1))
      )

2010年のデータは料金設定という大事なデータが欠損しているため、学習からは外すべし

## 料金区分price_am, price_pmが3以上のものを詳しく見てみる

In [ ]:
# 料金区分price_am, price_pmが3以上のものを詳しく見てみる
# そこまで数は多くなさそうなので全件表示
with pl.Config(tbl_rows=-1):
    print(
        train.filter(
            (pl.col("price_am") >= 3) | (pl.col("price_pm") >= 3)
        )
    )

3月、4月に高めの料金設定をしていることが見て取れる  

ただ一点、気になる部分あり  
│ id         ┆ datetime   ┆ y   ┆ y_ln     ┆ client ┆ price_am ┆ price_pm │  
│ 2011-12-30 ┆ 2011-12-30 ┆ 15  ┆ 2.70805  ┆ 0      ┆ 1        ┆ 3        │  
ってなってるけど、これは年末に一応営業したけど午後はスタッフ全然居ないから値付けを高くして調整しようとしたのか？

# トレンドを捉える(prophet)

In [ ]:
# 年を経るごとにyが上昇していくため、yの分布がどんどん右にシフトしていく問題に対処したい
# そこで、prophetによる時系列予測によってyのトレンドを予測し、yのトレンドを除去する

# polarsのdatetime型をpandasのdatetime型に変換する必要がある
## 訓練データ
train_pandas = (
    train
    .select(["datetime", "y"])
    .rename({"datetime": "ds", "y": "y"})
    .to_pandas().copy()
)
## テストデータ。datetimeカラムを渡すだけ
test_pandas = (
    test
    .select(["datetime"])
    .rename({"datetime": "ds"})
    .to_pandas().copy()
)

# 学習。prophetでは年を超えた全体的なトレンドだけを捉えたいので、seasonalityは全てFalseにする
model = Prophet(
    yearly_seasonality=False,
    weekly_seasonality=False,
    daily_seasonality=False,
    seasonality_mode='additive'
)
model.fit(train_pandas)

# 予測
# グラフ描画用
forecast_train = model.predict(train_pandas)
forecast_test = model.predict(test_pandas)

# 元データへの結合用
forecast_train_pl = pl.Series("y_trend", model.predict(train_pandas)["yhat"].values)

# trainの方にjoinして列追加
train = train.with_columns(forecast_train_pl)

In [ ]:
# 予測結果の可視化
fig, ax = plt.subplots(
    figsize=(12, 5),
    constrained_layout=True,
)
ax.set_title("Prophetによる予測（Train=青, Test=赤）")

# train側は Prophet の標準描画（青・点群付き）
model.plot(model.predict(train_pandas), ax=ax)

# test側は後から重ね描き（赤）
ax.plot(
    forecast_test["ds"], forecast_test["yhat"],
    color="red", linewidth=2, label="Test yhat"
)
ax.fill_between(
    forecast_test["ds"],
    forecast_test["yhat_lower"], forecast_test["yhat_upper"],
    color="red", alpha=0.2, label="Test interval"
) 

ax.legend()
plt.show()

# 後で再宣言しちゃいそうな変数はここで一度削除
del train_pandas, test_pandas, forecast_train, forecast_test, forecast_train_pl, model

コメント
- 全体的な上昇傾向が上手く捉えられている
- testデータの期間についても、尤もらしい傾向が出力できているように思う

## トレンドとの残差を確認

In [ ]:
# 残差列、曜日・月列を追加
train = train.with_columns(
    (pl.col("y") - pl.col("y_trend")).alias("y_resid"),
    pl.col("datetime").dt.weekday().alias("weekday"),
    pl.col("datetime").dt.month().alias("month")
)

In [ ]:
# トレンドとの差分を取ることで、年ごとの分布のズレが解消されるか確認する

fig, axes = plt.subplots(
    nrows=2, ncols=2,
    height_ratios=[1, 1], width_ratios=[2, 1],
    figsize=(12, 8),
    constrained_layout=True,
)
fig.suptitle("引越し数の推移と分布。元データ(上段)とトレンドとの残差(下段)の比較")

# 上段。そのまま
axes[0, 0].set_title("推移")
axes[0, 0].tick_params(axis="x", rotation=90)
sns.lineplot(data = train, x="datetime", y="y", hue="year",palette="deep", ax=axes[0, 0])
sns.lineplot(data = train, x="datetime", y="y_trend", color="0.1", ax=axes[0, 0])

axes[0, 1].set_title("分布")
sns.histplot(data = train, x="y", hue="year", palette="deep", ax=axes[0, 1])

# 下段。差分
axes[1, 0].set_title("推移(残差)")
axes[1, 0].tick_params(axis="x", rotation=90)
sns.scatterplot(data = train, x="datetime", y="y_resid", hue="year", palette="deep", marker="+", ax=axes[1, 0])

axes[1, 1].set_title("分布(残差)")
sns.histplot(data = train, x="y_resid", hue="year", palette="deep", ax=axes[1, 1])

plt.show()

コメント
- 狙い通り、うまい具合に年ごとの分布の中心が揃った。

In [ ]:
# 全体としての分布がどう変わったかも確認
skew_resid = stats.skew(train["y_resid"])
kurt_resid = stats.kurtosis(train["y_resid"])

fig, axes = plt.subplots(
    nrows=1, ncols=2,
    height_ratios=[1], width_ratios=[2, 1],
    figsize=(12, 4),
    constrained_layout=True,
)
fig.suptitle("引っ越し数yのトレンドとの残差の推移と分布")

# 下段。差分
axes[0].set_title("推移(残差)")
axes[0].tick_params(axis="x", rotation=90)
sns.scatterplot(data = train, x="datetime", y="y_resid", marker="+", ax=axes[0])

axes[1].set_title("分布(残差)")
sns.histplot(data = train, x="y_resid", ax=axes[1])
axes[1].text(0.6, 0.8,
                f"skewness={skew_resid:.2f}\nkurtosis={kurt_resid:.2f}",
                fontsize=12,
                transform=axes[1].transAxes)

plt.show()

コメント
- 分布の対称性は改善したが、正規分布に近づいたわけではない
- 上昇トレンドは排除出来たが、分散の増大傾向もある模様(不均一分散)。数値変換によって抑えてやる必要があるだろう

In [ ]:
# 曜日・月ごとの残差分布を確認
fig, axes = plt.subplots(
    nrows=1, ncols=2,
    height_ratios=[1],
    width_ratios=[1,1],
    figsize=(8, 4),
    constrained_layout=True,
)
# fig.suptitle("")

axes[0].set_title("曜日別の残差分布")
sns.boxplot(x="weekday", y="y_resid", data=train, ax=axes[0])

axes[1].set_title("月別の残差分布")
sns.boxplot(x="month", y="y_resid", data=train, ax=axes[1])

plt.show()

コメント
- 火・水・木は受注減りがち。土日は受注多め
- 繁忙期である3~4月は確かに通常期よりも受注が多い

# 数値変換を試す

lightGBMのようなノンパラメトリックなモデルでは分布の形はあまり関係ないが、  
それでも分散が増大していく傾向は抑えないといけない

In [ ]:
# 数値変換の定義（カラム名, 変換式）
transforms = {
    "y_ln": pl.col("y").log(),
    "y_bc_1_2": (pl.col("y").pow(1/2) - 1) / (1/2),
    "y_bc_1_10": (pl.col("y").pow(1/10) - 1) / (1/10),
}

moments = {}   # 変換ごとの skew / kurtosis を格納
results = {}   # 変換ごとの prophet による予測結果を保存

for name, expr in transforms.items():
    # 変換列をtrainに追加
    train = train.with_columns(expr.alias(name))

    # prophet用にpandas化したものを用意
    df = (
        train
        .select(["datetime", name])
        .rename({"datetime": "ds", name: "y"})
        .to_pandas()
        .copy()
    )

    # Prophet モデル学習
    model = Prophet(
        yearly_seasonality=False,
        weekly_seasonality=False,
        daily_seasonality=False,
        seasonality_mode="additive",
    )
    model.fit(df)

    # 予測結果を保存
    results[name] = model.predict(df)
    result_pl = pl.Series(f"{name}_trend", results[name]["yhat"].values)

    # trainに予測結果を追加
    train = train.with_columns(result_pl)
    
    # 残差列を追加
    train = train.with_columns(
        (pl.col(name) - pl.col(f"{name}_trend")).alias(f"{name}_resid")
    )

    # 統計量を計算したい列を選択
    x = train[f"{name}_resid"].to_numpy()
    # bias=False で標本推定、fisher=True で excess（正規=0）
    moments[name] = {
        "skew": float(stats.skew(x, bias=True)),
        "kurt": float(stats.kurtosis(x, fisher=True, bias=True)),
    }

In [ ]:
# 描画

fig, axes = plt.subplots(
    nrows=len(transforms), ncols=2,
    height_ratios=[1, 1, 1], width_ratios=[2, 1],
    figsize=(15, 4 * len(transforms)),
    constrained_layout=True,
)
fig.suptitle("各種数値変換を行った y の、トレンドとの残差の推移と分布", fontsize=16)

# ループで各変換を可視化
for i, name in enumerate(transforms.keys()):
    resid_col = f"{name}_resid"

    # --- 残差の推移 ---
    axes[i, 0].set_title(f"推移 ({name})")
    sns.lineplot(
        data=train.to_pandas(),
        x="datetime", y=resid_col,
        ax=axes[i, 0], linewidth=1
    )
    axes[i, 0].axhline(0, color="red", linestyle="--", linewidth=1)

    # --- 残差の分布 ---
    axes[i, 1].set_title(f"分布 ({name})")
    sns.histplot(
        data=train.to_pandas(),
        x=resid_col,
        kde=True, stat="density",
        ax=axes[i, 1],
    )
    axes[i, 1].axvline(0, color="red", linestyle="--", linewidth=1)

    # skew / kurt を表示
    m = moments[name]
    axes[i, 1].text(
        0.05, 0.95,
        f"skew={m['skew']:.2f}\nkurt={m['kurt']:.2f}",
        transform=axes[i, 1].transAxes,
        ha="left", va="top",
        fontsize=11, color="black",
        bbox=dict(boxstyle="round", facecolor="white", alpha=0.7),
    )

plt.show()

グラフについてのコメント
- どれも数値変換前の歪度・尖度と比べると正規分布に近づいたが、もう少しいい感じに変換できる設定はないだろうか？
  - TODO: 自動でλ等の最適な値を計算してくれる方法があった気がする
- とはいえ、分散の増大を抑えたいだけならどれでも良いだろう

独り言
- polars↔pandasの行ったり来たりが非常に面倒。可視化が絡むEDAでは、polarsを使っていることが非常に重い足枷になる印象

## 没：y_residを数値変換→分析といった順番でやった版

In [ ]:
# y_residを数値変換→残差分析といった順番でやった版。尖度・歪度がおかしくなったので没

# # 数値変換の定義（カラム名, 変換式）
# transforms = {
#     "y_resid_ln": pl.col("y_resid").log(),
#     "y_resid_bc_1_2": (pl.col("y_resid").pow(1/2) - 1) / (1/2),
#     "y_resid_bc_1_10": (pl.col("y_resid").pow(1/10) - 1) / (1/10),
# }

# moments = {}   # 変換ごとの skew / kurtosis を格納
# results = {}   # 変換ごとの prophet による予測結果を保存

# for name, expr in transforms.items():
#     # 変換列をtrainに追加
#     train = train.with_columns(expr.alias(name))

#     # 統計量を計算
#     x = train[name].to_numpy()
#     # bias=False で標本推定、fisher=True で excess（正規=0）
#     moments[name] = {
#         "skew": float(stats.skew(x, bias=False)),
#         "kurt": float(stats.kurtosis(x, fisher=True, bias=False)),
#     }

# # 描画設定
# fig, axes = plt.subplots(
#     nrows=len(transforms), ncols=2,
#     figsize=(15, 4 * len(transforms)),
#     constrained_layout=True,
# )
# fig.suptitle("各種数値変換を行った y の、トレンドとの残差の推移と分布", fontsize=16)

# # ループで各変換を可視化
# for i, name in enumerate(transforms.keys()):
#     # --- 残差の推移 ---
#     axes[i, 0].set_title(f"推移 ({name})")
#     sns.lineplot(
#         data=train.to_pandas(),
#         x="datetime", y=name,
#         ax=axes[i, 0], linewidth=1
#     )
#     axes[i, 0].axhline(0, color="red", linestyle="--", linewidth=1)

#     # --- 残差の分布 ---
#     axes[i, 1].set_title(f"分布 ({name})")
#     sns.histplot(
#         data=train.to_pandas(),
#         x=name,
#         kde=True, stat="density",
#         ax=axes[i, 1],
#     )
#     axes[i, 1].axvline(0, color="red", linestyle="--", linewidth=1)

#     # skew / kurt を表示
#     m = moments[name]
#     axes[i, 1].text(
#         0.05, 0.95,
#         f"skew={m['skew']:.2f}\nkurt={m['kurt']:.2f}",
#         transform=axes[i, 1].transAxes,
#         ha="left", va="top",
#         fontsize=11, color="black",
#         bbox=dict(boxstyle="round", facecolor="white", alpha=0.7),
#     )

# plt.show()

# EDAのまとめ
1. 目的変数yが増加傾向のため、年毎の分布が右にシフトしている。
    - to do: 年毎の目的変数yの分布を揃える必要がある。  
2. 目的変数yは繁忙期(3月、4月)とそれ以外の時期で分布が異なる
3. 休業日close = 1は目的変数y = 0のため、休業日を分離した。
4. 目的変数y = 0の分布の右側の裾野が広いので対数変換で裾野を狭めた。
5. 法人対応clientを2014年度から始めた
6. 法人が絡む特殊な引越しを行う場合(close = 1)、目的変数の平均値が約9増える
7. 料金区分(price_amとprice_pm)は、繁忙期に高い価格を設定している
8. 2010年の料金区分が全て欠測値のため削除した
9. 午後の料金区分は午前よりも安い傾向にある。

# 参考サイト  
[コンペサイト アップル 引越し需要予測](https://signate.jp/competitions/269/data)  
[SIGNATE SOTA アップル 引越し需要予測 備忘録](https://zenn.dev/tremendous1192/articles/ea6e73359ee764)  


# htmlに変換したものを出力

In [ ]:
from datetime import datetime
from pathlib import Path

base_name = "EDA"
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_dir = Path("html")
output_file = f"{base_name}_{timestamp}"

!jupyter nbconvert --to html EDA.ipynb --output "{output_file}" --output-dir "{output_dir}"